# Big Data 3 - Spark SQL

In [1]:
%login montana.dataapplab.com

[ssh] Login to montana.dataapplab.com...
[ssh] host=montana.dataapplab.com hostname=montana.dataapplab.com other_conf={'user': 'dcdsdepeizhang', 'port': 49233, 'keyfile': ['/home/dz3vg/.ssh/id_rsa'], 'load_system_ssh_config': False, 'missing_host_policy': <paramiko.client.WarningPolicy object at 0x7f7e8320bbd0>}
[ssh] forwarding local agent
[ssh] Successfully logged in.


In [2]:
pwd

[ssh] host = montana.dataapplab.com, cwd = /home/dcdsdepeizhang
/home/dcdsdepeizhang


In [3]:
ls -l

[ssh] host = montana.dataapplab.com, cwd = /home/dcdsdepeizhang
total 8
drwxr-xr-x. 3 dcdsdepeizhang student 4096 Jul 30 21:30 data
drwxr-xr-x. 4 dcdsdepeizhang student 4096 Jul 30 21:01 demo


In [4]:
cd demo/MapReduce/

[ssh] host = montana.dataapplab.com, cwd = /home/dcdsdepeizhang
[ssh] new cwd: /home/dcdsdepeizhang/demo/MapReduce


In [5]:
ls -l

[ssh] host = montana.dataapplab.com, cwd = /home/dcdsdepeizhang/demo/MapReduce
total 16
-rw-r--r--. 1 dcdsdepeizhang student   37 Jul 30 22:59 data.txt
-rw-r--r--. 1 dcdsdepeizhang student  539 Jul 30 22:59 mapper.py
-rw-r--r--. 1 dcdsdepeizhang student 1041 Jul 30 22:59 reducer.py
-rw-r--r--. 1 dcdsdepeizhang student   38 Jul 30 23:00 result.txt


In [6]:
cat << EOF > mapper.py
#!/usr/bin/env python

import sys

# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    # split the line into words
    words = line.split()
    # increase counters
    for word in words:
        # write the results to STDOUT (standard output);
        # what we output here will be the input for the
        # Reduce step, i.e. the input for reducer.py
        # 
        # tab-delimited; the trivial word count is 1
        print '%s\t%s' % (word, 1)

EOF

[ssh] host = montana.dataapplab.com, cwd = /home/dcdsdepeizhang/demo/MapReduce


In [7]:
cat << EOF > reducer.py
#!/usr/bin/env python

from operator import itemgetter
import sys

current_word = None
current_count = 0
word = None

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    
    # parse the input we got from mapper.py
    word, count = line.split('\t', 1)
    
    # convert count (currently a string) to int
    try:
        count = int(count)
    except ValueError:
        # count was not a number, so silently
        # ignore/discard this line
        continue
    
    # this IF-switch only works because Hadoop sorts map output
    # by key (here: word) before it is passed to the reducer
    if current_word == word:
        current_count += count
    else:
        if current_word:
            # write result to STDOUT
            print '%s\t%s' % (current_word, current_count)
        current_count = count
        current_word = word
 
# do not forget to output the last word if needed!
if current_word == word:
    print '%s\t%s' % (current_word, current_count)

EOF

[ssh] host = montana.dataapplab.com, cwd = /home/dcdsdepeizhang/demo/MapReduce


In [8]:
cat << EOF > data.txt
this is a test
this is a map reduce test
test test test test test

EOF

[ssh] host = montana.dataapplab.com, cwd = /home/dcdsdepeizhang/demo/MapReduce


In [9]:
ls -l

[ssh] host = montana.dataapplab.com, cwd = /home/dcdsdepeizhang/demo/MapReduce
total 16
-rw-r--r--. 1 dcdsdepeizhang student   67 Jul 30 23:03 data.txt
-rw-r--r--. 1 dcdsdepeizhang student  539 Jul 30 23:03 mapper.py
-rw-r--r--. 1 dcdsdepeizhang student 1041 Jul 30 23:03 reducer.py
-rw-r--r--. 1 dcdsdepeizhang student   38 Jul 30 23:00 result.txt


In [10]:
hdfs dfs -rm hdfs:///user/dcdsdepeizhang/data/data.txt

[ssh] host = montana.dataapplab.com, cwd = /home/dcdsdepeizhang/demo/MapReduce
20/07/30 23:03:42 INFO fs.TrashPolicyDefault: Moved: 'hdfs://m1.mt.dataapplab.com:8020/user/dcdsdepeizhang/data/data.txt' to trash at: hdfs://m1.mt.dataapplab.com:8020/user/dcdsdepeizhang/.Trash/Current/user/dcdsdepeizhang/data/data.txt


In [11]:
hdfs dfs -put data.txt hdfs:///user/dcdsdepeizhang/data/

[ssh] host = montana.dataapplab.com, cwd = /home/dcdsdepeizhang/demo/MapReduce


In [12]:
hdfs dfs -ls hdfs:///user/dcdsdepeizhang/data/

[ssh] host = montana.dataapplab.com, cwd = /home/dcdsdepeizhang/demo/MapReduce
Found 2 items
-rw-r--r--   3 dcdsdepeizhang student         67 2020-07-30 23:03 hdfs:///user/dcdsdepeizhang/data/data.txt
-rw-r--r--   3 dcdsdepeizhang student       5107 2020-07-30 21:30 hdfs:///user/dcdsdepeizhang/data/iris.csv


In [13]:
hdfs dfs -rm -r -f hdfs:///user/dcdsdepeizhang/temp/

[ssh] host = montana.dataapplab.com, cwd = /home/dcdsdepeizhang/demo/MapReduce
20/07/30 23:03:53 INFO fs.TrashPolicyDefault: Moved: 'hdfs://m1.mt.dataapplab.com:8020/user/dcdsdepeizhang/temp' to trash at: hdfs://m1.mt.dataapplab.com:8020/user/dcdsdepeizhang/.Trash/Current/user/dcdsdepeizhang/temp


In [14]:
hdfs dfs -mkdir hdfs:///user/dcdsdepeizhang/temp/

[ssh] host = montana.dataapplab.com, cwd = /home/dcdsdepeizhang/demo/MapReduce


In [15]:
hadoop jar /usr/hdp/2.5.3.0-37/hadoop-mapreduce/hadoop-streaming-2.7.3.2.5.3.0-37.jar \
-file /home/dcdsdepeizhang/demo/MapReduce/mapper.py -mapper /home/dcdsdepeizhang/demo/MapReduce/mapper.py \
-file /home/dcdsdepeizhang/demo/MapReduce/reducer.py -reducer /home/dcdsdepeizhang/demo/MapReduce/reducer.py \
-input /user/dcdsdepeizhang/data/data.txt -output /user/dcdsdepeizhang/temp/result

[ssh] host = montana.dataapplab.com, cwd = /home/dcdsdepeizhang/demo/MapReduce
20/07/30 23:03:59 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/home/dcdsdepeizhang/demo/MapReduce/mapper.py, /home/dcdsdepeizhang/demo/MapReduce/reducer.py] [/usr/hdp/2.5.3.0-37/hadoop-mapreduce/hadoop-streaming-2.7.3.2.5.3.0-37.jar] /tmp/streamjob4052655824797882179.jar tmpDir=null
20/07/30 23:04:02 INFO impl.TimelineClientImpl: Timeline service address: http://m2.mt.dataapplab.com:8188/ws/v1/timeline/
20/07/30 23:04:02 INFO client.RMProxy: Connecting to ResourceManager at m2.mt.dataapplab.com/192.168.1.52:8050
20/07/30 23:04:02 INFO client.AHSProxy: Connecting to Application History server at m2.mt.dataapplab.com/192.168.1.52:10200
20/07/30 23:04:02 INFO impl.TimelineClientImpl: Timeline service address: http://m2.mt.dataapplab.com:8188/ws/v1/timeline/
20/07/30 23:04:02 INFO client.RMProxy: Connecting to ResourceManager at m2.mt.dataapplab

In [16]:
hdfs dfs -ls hdfs:///user/dcdsdepeizhang/temp/result/

[ssh] host = montana.dataapplab.com, cwd = /home/dcdsdepeizhang/demo/MapReduce
Found 2 items
-rw-r--r--   3 dcdsdepeizhang student          0 2020-07-30 23:04 hdfs:///user/dcdsdepeizhang/temp/result/_SUCCESS
-rw-r--r--   3 dcdsdepeizhang student         38 2020-07-30 23:04 hdfs:///user/dcdsdepeizhang/temp/result/part-00000


In [17]:
#hdfs dfs -get hdfs:///user/dcdsdepeizhang/temp/result/part-00000 result.txt
hdfs dfs -getmerge hdfs:///user/dcdsdepeizhang/temp/result/ result.txt

[ssh] host = montana.dataapplab.com, cwd = /home/dcdsdepeizhang/demo/MapReduce


In [18]:
head -20 result.txt

[ssh] host = montana.dataapplab.com, cwd = /home/dcdsdepeizhang/demo/MapReduce
a	2
is	2
map	1
reduce	1
test	7
this	2


In [19]:
%logout

[ssh] Closing existing connection.
[ssh] Successfully logged out.
